In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import json
import contextily as ctx
import warnings
warnings.filterwarnings("ignore")

In [5]:
LBCS = {
    'SITIO ERIAZO': 9000,
    'BIENES COMUNES': 9999,
    'OTROS': 9200,
    'SALUD': 6500,
    'CULTO': 6600,
    'BODEGA Y ALMACENAJE': 3120,
    'HABITACIONAL': 1100,
    'OFICINA': 2300,
    'COMERCIO': 2100,
    'EDUCACION Y CULTURA': 4100,
    'INDUSTRIA': 3000,
    'AREAS VERDES': 7000,
    'PRE ROL': 9999,
    'TRANSPORTE Y TELEC': 4000, #434public0 Telecomunicaciones
    'DEPORTE Y RECREACION': 7100,
    'HOTEL, MOTEL': 1200,
    'ESTACIONAMIENTO': 5210,
    'ADM PUBLICA Y DEFENSA': 6200, #+6300
    'MULTI ROL': 9999,
    'AGRICOLA': 9000,
}

LBCS_translated = {
    'SITIO ERIAZO': 'Vacant lot',
    'BIENES COMUNES': 'Public spaces',
    'OTROS': 'Other',
    'SALUD': 'Healthcare',
    'CULTO': 'Religious',
    'BODEGA Y ALMACENAJE': 'Warehouse and storage',
    'HABITACIONAL': 'Residential',
    'OFICINA': 'Office',
    'COMERCIO': 'Commercial',
    'EDUCACION Y CULTURA': 'Education and culture',
    'INDUSTRIA': 'Industry',
    'AREAS VERDES': 'Park',
    'TRANSPORTE Y TELEC': 'Transportation and telecommunications', #4340 Telecommunications
    'DEPORTE Y RECREACION': 'Sports and recreation',
    'HOTEL, MOTEL': 'Hotel, motel',
    'ESTACIONAMIENTO': 'Parking',
    'ADM PUBLICA Y DEFENSA': 'Public administration and defense', #+6300
    'AGRICOLA': 'Agricultural',
}

In [2]:
grid = gpd.read_parquet('/app/data/new_grid_cityscope.parquet')

In [7]:
grid['name'] = grid['name'].map(LBCS_translated)